In [ ]:
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
import tensorflow as tf


In [ ]:
csv_file = "pairs.csv"
csv_data = pd.read_csv(csv_file, low_memory = False)#防止弹出警告
pairs = pd.DataFrame(csv_data)
pairs.columns =['s','bug1','bug2','label']
part_8 = pairs.sample(frac = 0.8)
part_2 = pairs.drop(part_8.index)
train =[part_8['bug1'].values,part_8['bug2'].values]
test =[part_2['bug1'].values,part_2['bug2'].values]
train_label = part_8['label'].values
test_label = part_2['label'].values
train_label = np.where(train_label == -1, 0, train_label)
test_label = np.where(test_label == -1, 0, test_label)
tset = [pairs['bug1'].values,pairs['bug2'].values]

In [79]:

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")
input_ids = []
attention_masks = []
for sent in train[0]:
 
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0) #句子token
attention_masks = torch.cat(attention_masks, dim=0)#分句标志
tokens_tensor = torch.tensor(input_ids)
segments_tensors = torch.tensor(attention_masks)
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True)        
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]
token_embeddings0 = torch.stack(hidden_states, dim=0)

input_ids = []
attention_masks = []
for sent in train[1]:
 
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0) #句子token
attention_masks = torch.cat(attention_masks, dim=0)#分句标志
tokens_tensor = torch.tensor(input_ids)
segments_tensors = torch.tensor(attention_masks)
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True)        
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]
token_embeddings1 = torch.stack(hidden_states, dim=0)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max 

In [100]:
left = tf.convert_to_tensor(torch.sum(token_embeddings1[-4:], dim=0))
right = tf.convert_to_tensor(torch.sum(token_embeddings0[-4:], dim=0))


In [104]:
Labels = tf.one_hot(train_label, depth=1)

In [106]:
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.models import Model

# 定义输入张量形状
input_shape = (64, 768)

# 定义卷积层参数
filters = 64
kernel_size = 3
activation = 'relu'
pool_size = 2

# 定义左侧子网络
left_input = Input(input_shape)
x = Conv1D(filters, kernel_size, activation=activation)(left_input)
x = MaxPooling1D(pool_size)(x)
x = Conv1D(filters, kernel_size, activation=activation)(x)
x = MaxPooling1D(pool_size)(x)
x = Flatten()(x)
x = Dense(128, activation=activation)(x)
x = Dropout(0.2)(x)
left_output = Dense(1, activation='sigmoid')(x)

# 定义右侧子网络，其结构与左侧子网络相同
right_input = Input(input_shape)
x = Conv1D(filters, kernel_size, activation=activation)(right_input)
x = MaxPooling1D(pool_size)(x)
x = Conv1D(filters, kernel_size, activation=activation)(x)
x = MaxPooling1D(pool_size)(x)
x = Flatten()(x)
x = Dense(128, activation=activation)(x)
x = Dropout(0.2)(x)
right_output = Dense(1, activation='sigmoid')(x)

# 定义孪生网络模型
model = Model(inputs=[left_input, right_input], outputs=[left_output, right_output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.summary()



epochs = 100 # 训练轮数
batch_size = 32 # 批次大小
model.fit([left,right], Labels, epochs=epochs, batch_size=batch_size)



Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_35 (InputLayer)          [(None, 64, 768)]    0           []                               
                                                                                                  
 input_36 (InputLayer)          [(None, 64, 768)]    0           []                               
                                                                                                  
 conv1d_32 (Conv1D)             (None, 62, 64)       147520      ['input_35[0][0]']               
                                                                                                  
 conv1d_34 (Conv1D)             (None, 62, 64)       147520      ['input_36[0][0]']               
                                                                                           

KeyboardInterrupt: ignored